<a href="https://colab.research.google.com/github/Dukkipati-Likithasree/FeynnLabs-project/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error

# Function to fetch historical stock data from Yahoo Finance
def fetch_stock_data(symbol, start_date, end_date):
    df = yf.download(symbol, start=start_date, end=end_date)
    return df

# Function to preprocess the data
def preprocess_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data.reshape(-1, 1))
    return scaled_data, scaler

# Function to create dataset for LSTM
def create_dataset(data, time_step):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

# Define parameters
symbol = 'AAPL'  # Example stock symbol (Apple Inc.)
start_date = '2010-01-01'
end_date = '2022-01-01'
time_step = 60  # Number of timesteps to look back

# Fetching stock data
raw_data = fetch_stock_data(symbol, start_date, end_date)
close_prices = raw_data['Close'].values

# Preprocessing data
scaled_data, scaler = preprocess_data(close_prices)

# Creating train-test split
train_size = int(len(scaled_data) * 0.8)
test_size = len(scaled_data) - train_size
train_data, test_data = scaled_data[0:train_size, :], scaled_data[train_size:len(scaled_data), :]

# Creating dataset for LSTM
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Building LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Training the model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Making predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

# Evaluating the model
mse = mean_squared_error(close_prices[train_size + time_step + 1:], predictions)
print("Mean Squared Error:", mse)


[*********************100%%**********************]  1 of 1 completed


Epoch 1/100
74/74 [==============================] - 17s 71ms/step - loss: 0.0013
Epoch 2/100
74/74 [==============================] - 4s 48ms/step - loss: 1.9750e-04
Epoch 3/100
74/74 [==============================] - 4s 47ms/step - loss: 1.9064e-04
Epoch 4/100
74/74 [==============================] - 4s 59ms/step - loss: 1.8528e-04
Epoch 5/100
74/74 [==============================] - 4s 51ms/step - loss: 1.5406e-04
Epoch 6/100
74/74 [==============================] - 4s 48ms/step - loss: 1.6052e-04
Epoch 7/100
74/74 [==============================] - 4s 50ms/step - loss: 1.3372e-04
Epoch 8/100
74/74 [==============================] - 4s 60ms/step - loss: 1.3266e-04
Epoch 9/100
74/74 [==============================] - 4s 47ms/step - loss: 1.2295e-04
Epoch 10/100
74/74 [==============================] - 4s 48ms/step - loss: 1.1574e-04
Epoch 11/100
74/74 [==============================] - 5s 63ms/step - loss: 1.0865e-04
Epoch 12/100
74/74 [==============================] - 3s 47ms/step